In this unit, we describe the procedure for training the commuter activity classifier. Further, we present the classification result for commuter activity classifier.

# Commuter activity classifier

In [1]:
'''Import and initialize MongoClient'''
import subprocess
import os
import sys
import json
from pymongo import     MongoClient
con = MongoClient()
from scipy import integrate
import numpy as np
import pprint
import math
from pathlib import Path

In [2]:
'''Import project specific library'''
sys.path.append(os.path.join(os.getcwd(), 'LibCode'))
import ReadAcclGPSRecord
import SegmentsOtherThanStoppageSegments
import EarthaxisAcceleration
import EarthaxisAccelerationOnRawRecords
import ComputeFeaturesTransportMode
import FeaturesExtraction
import TransportModeFeatureHelper
import CAC_Helper

In [3]:
'''Import Libs'''
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix,precision_score, recall_score, f1_score
from sklearn import tree
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from collections import Counter
from sklearn.impute import SimpleImputer

In [4]:
'''For updating the lib changes effects'''
import importlib
importlib.reload(ReadAcclGPSRecord)
importlib.reload(SegmentsOtherThanStoppageSegments)
importlib.reload(EarthaxisAcceleration)
importlib.reload(EarthaxisAccelerationOnRawRecords)
importlib.reload(ComputeFeaturesTransportMode)
importlib.reload(FeaturesExtraction)
importlib.reload(TransportModeFeatureHelper)
importlib.reload(CAC_Helper)

<module 'CAC_Helper' from '/home/pruthvish/JRF/GitVersion_APTS_Software_Np/code/LibCode/CAC_Helper.py'>

## Save in MongoDB
The commuter activity classifier is trained and validated using the trip records of the learning module. Aforementioned, the learning module application is carried by the data colleciton volunteers on their trips on different types of vehicles. The accelerometer record is stored in the `RouteName = HAR_PDPU_SANAND` database of MongoDB. 

In [5]:
def ReadAndSaveData(SitStandRecordDir, RouteName):
    
    '''
    input: The route name and dataset directory name
    
    output: None
    
    function: It fetches the data records placed in the specified data set directory 
    and save them in the MongoDB database
    '''    
    
    '''E.g., BM, CM, Bike, Car'''
    for fileName in [f for f in os.listdir(os.path.join(SitStandRecordDir,RouteName))]:
        fileNameList = fileName.split('_')
        #print(fileName)
        ReadAcclGPSRecord.SaveInMongoFunction(RouteName,
                                              os.path.join(SitStandRecordDir,RouteName,fileName),
                                              fileNameList[0]+'.'+fileNameList[1]+'.'+fileNameList[2]+'.'+fileNameList[3])


## Decomposition of accelerometer data
The accelerometer record data is decomposed to `horizontal and vertical components` with the `IntervalLength` of `140 points.`

In [6]:
def ComputeEarthAxisComponentRaw(RouteName, IntervalLength, RecordType):
    
    '''
    input: The route name, interval length for computing accelerometer components
    and record type variable to specify the segment type of the records.
    
    output: None
    
    function: It extracts the raw accelerometer records from the MongoDB database
    for the entire trip records and executes the code to compute the horizontal and
    vertical components based on the method proposed in Jigsaw paper. 
    '''    
    
    '''To EarthAxis'''
    SingleTripsInfo = [LR['SingleTripInfo'] for LR in 
                       con[RouteName]['TripsInfo'].find({'ConvertedToEarthAxis':False})]

    for SingleTripInfo in SingleTripsInfo:
        print(SingleTripInfo)

        AcclMagRecord = [collection for collection in 
                         con[RouteName][SingleTripInfo+'.AcclMagData.Raw'].find().sort([('GPSIndex',1)])]

        AcclMagRecord = CAC_Helper.GetSlicedAcclMagRecord(RouteName,SingleTripInfo)

        EarthaxisAccelerationOnRawRecords.ProcessEarthaxisHVComponentUsingJigSawMethod (RouteName,
                                                                                        SingleTripInfo,
                                                                                        AcclMagRecord,
                                                                                        IntervalLength,RecordType)
        con[RouteName]['TripsInfo'].update_one(
            {'SingleTripInfo':SingleTripInfo},{'$set':{'ConvertedToEarthAxisRaw':True}})


## Feature extraction
The feature `set-1 to set-4` are computed using the orientation independent horizontal and vertical components of accelerometer records for the window of `128 samples and 50% overlap`.

In [7]:
def GetMode(SingleTripInfo):

    '''
    input: The trip name
    output: The mode of the trip name
    function: It extracts the mode (label) of the trip records
    '''
    
    if (SingleTripInfo.split('.')[1] == 'Seating' or SingleTripInfo.split('.')[1] == 'Sitting') and (SingleTripInfo.split('.')[2]=='Hand'):
        Mode = 0
        
    elif (SingleTripInfo.split('.')[1] == 'Seating' or SingleTripInfo.split('.')[1] == 'Sitting') and (SingleTripInfo.split('.')[2]=='ShirtPocket'):
        Mode = 1
        
    elif (SingleTripInfo.split('.')[1] == 'Standing') and (SingleTripInfo.split('.')[2]=='ShirtPocket'):
        Mode = 2

    elif (SingleTripInfo.split('.')[1] == 'Standing') and (SingleTripInfo.split('.')[2]=='TrouserPocket'):
        Mode = 3
                
    elif (SingleTripInfo.split('.')[1] == 'Standing') and (SingleTripInfo.split('.')[2]=='Hand'):
        Mode = 4
        
    return(Mode)

In [8]:
def ComputeFeatures_Raw(ProjectDataUsed, WindowList, WindowIndex, RecordType, RouteName):
    
    '''
    input: The ProjectDataUsed variable specify whether the project dataset should to use project 
    dataset or the user collected dataset. The other variables specify the window size for feature
    computation, record type (Raw or Segment other than stoppage segment), and route name.
    
    output: None
    
    function: It extracts the appropriate accelerometer components from the MongoDB database based
    on the provided input and computes the features on the windowed accelerometer component. The 
    computed features are stored in the MongoDB database.
    '''
    
    SingleTripsInfo = [LR['SingleTripInfo'] for LR in 
                       con[RouteName]['TripsInfo'].find({'FeaturesExtracted':False})]
    for SingleTripInfo in SingleTripsInfo:
        print(SingleTripInfo)
        HVRecord = [Rec for Rec in 
                    con[RouteName][SingleTripInfo+'.EAccHVComponent'+RecordType].find().sort([('GPSIndex',1)])]

        #print(HVRecord[0])
        #input()
        Mode = GetMode(SingleTripInfo)

        FeaturesExtraction.ComputeFeatureForComponents(RouteName,HVRecord,Mode,
                                                       WindowSize,SingleTripInfo,RecordType)

        ComputeFeaturesTransportMode.ComputeFeature(SingleTripInfo,HVRecord,Mode,WindowSize,
                                                    RecordType, RouteName)

        con[RouteName]['TripsInfo'].update_one({'SingleTripInfo':SingleTripInfo},
                                               {'$set':{'FeaturesExtracted':True}})

        
    '''
    if ProjectDataUsed==True:
    

    else:
        Trips = SingleTripsInfo = [LR['SingleTripInfo'] for LR in con[RouteName]['TripsInfo'].find(
            {'ConvertedToEarthAxisRaw':True})]

        BusTrips = [Trip for Trip in Trips if Trip.split['_'][1]=='Bus']
        CarTrips = [Trip for Trip in Trips if Trip.split['_'][1]=='Car']
        BikeTrips = [Trip for Trip in Trips if Trip.split['_'][1]=='Bike']

        TransportModeFeatureHelper.GetFeaturesForGivenTripType (BusTrips, 0,
                                                                WindowList[WindowIndex],RecordType, RouteName)
        TransportModeFeatureHelper.GetFeaturesForGivenTripType (CarTrips, 1,
                                                                WindowList[WindowIndex],RecordType, RouteName)
        TransportModeFeatureHelper.GetFeaturesForGivenTripType (BikeTrips, 2,
                                                                WindowList[WindowIndex],RecordType, RouteName)
    '''        

## Processing: For Records other than Stoppage Segment
This section describes the procedure for training the classifier on the `segments other than stoppage segment`.

## Decomposition of accelerometer data
The accelerometer record data is decomposed to `horizontal and vertical components` with the `IntervalLength` of `140 points.`

In [9]:
def ExtractSegment_EarthAxis_OtherThanStoppages(RouteName, Speed_H = 5.6, Speed_L = 3):
    
    '''
    input: The route name, interval length for computing accelerometer components, 
    speed range for computing stoppage segments and record type variable to specify
    the segment type of the records.
    
    output: None
    
    function: It extracts the raw accelerometer records from the MongoDB database
    for the computing stoppage segments and executes the code to compute the horizontal 
    and vertical components based on the method proposed in Jigsaw paper. 
    '''
        
    
    '''Extract segment other than stoppage segments'''
    if ProjectDataUsed==True:
        BMSingleTripsInfoList,CMSingleTripsInfoList,PDSingleTripsInfoList = SegmentsOtherThanStoppageSegments.GetTripsBasedOnType(RouteName)

        SegmentsOtherThanStoppageSegments.GetBMAndPDSegments(RouteName,BMSingleTripsInfoList,Speed_H, Speed_L)
        SegmentsOtherThanStoppageSegments.GetBMAndPDSegments(RouteName,PDSingleTripsInfoList,Speed_H, Speed_L)
        SegmentsOtherThanStoppageSegments.GetCMSegments(RouteName,CMSingleTripsInfoList,BMSingleTripsInfoList)
        SegmentsOtherThanStoppageSegments.GetGPSAndAcclReadOfSegment(RouteName)

    else:
        SingleTripsInfo = [LR['SingleTripInfo'] for LR in con[RouteName]['TripsInfo'].find({'SegmentExtracted':False})]
        SegmentsOtherThanStoppageSegments.GetBMAndPDSegments(RouteName,SingleTripsInfo, Speed_H, Speed_L)
        SegmentsOtherThanStoppageSegments.GetGPSAndAcclReadOfSegment(RouteName)

    '''
    for SingleTripInfo in SingleTripsInfo:
        con[RouteName]['TripsInfo'].update_one(
            {'SingleTripInfo':SingleTripInfo},{'$set':{'ConvertedToEarthAxis':False}})
    '''
    
    EarthaxisAcceleration.ConvertToEarthaxisAcc(RouteName,RecordType)

In [10]:
def ComputeFeatures_SegmentOtherThanStoppages_SitStand(ProjectDataUsed, WindowList, 
                                                       WindowIndex, RecordType, RouteName):
    
    '''
    input: 
        ProjectDataUsed: flag to determine whether the project dataset or user dataset is used
        WindowList and WindowIndex: The window length for feature computation
        RecordType: The flag to determine whether the features of entire trip is used (.Raw) or
        the features of segment other than stoppage segments is used (.SegmentOtherThanStoppage).
        
    output: None
    
    function: It computes the features for the segment other than stoppage segments for the 
    provided route name.
    '''
    
    '''Recent Attempt'''
    Trips = [LR['SingleTripInfo'] for LR in con[RouteName]['TripsInfo'].find(
        {'ConvertedToEarthAxis':True})]

    #Mode = GetMode(Trips)
    
    TransportModeFeatureHelper.ExtractFeaturesOfGivenTypeOfTrip(Trips,"Bus",0,
                                                                WindowList[WindowIndex],RecordType, RouteName)


## For saving Mongo data in numpy

In [11]:
def LoadInMongoFromNp(RouteName, NpPathDir):
    
    '''
    input: The route name and numpy directory path
    output: The MongoDB database collections from the Numpy files
    function: It creates the MongoDB database of TripsInfo collections 
    and features collections from the Numpy files.
    '''    
    
    CollectionName = 'TripsInfo'
    TripsInfoRecords = np.load(f'{NpPathDir}/{RouteName}/{CollectionName}.npy', allow_pickle=True)
    
    print('Saving data in mongoDB')
    print(RouteName, CollectionName)
    con[RouteName][CollectionName].insert_many(TripsInfoRecords.tolist())
    
    CollectionNames = os.listdir(f'{NpPathDir}/{RouteName}')
    #print(CollectionNames)
    
    CollectionNames_1 = [rec for rec in CollectionNames if '\'.' not in rec] # To address the error
    CollectionNames_2 = [rec for rec in CollectionNames if 'Feature' in rec]
    
    #print('Saving data in mongoDB')
    for Collection in CollectionNames_2:
        RecordsList = np.load(f'{NpPathDir}/{RouteName}/{Collection}', allow_pickle=True)
        #print(Collection)
        #pprint.pprint(RecordsList[0:3])
        
        CollectionName = Collection[0:-4]
        
        print(RouteName, CollectionName)
        con[RouteName][CollectionName].insert_many(RecordsList.tolist())
        
    
    
def SaveInNp(RouteName, NpPathDir):
    
    '''
    input: The route name and numpy directory path
    output: Numpy files of the MongoDB database
    function: It stores the Numpy files for the MongoDB database in the specified directory path
    '''
        
    
    CollectionNames = [Collection for Collection in 
                        con[RouteName].list_collection_names() if Collection!='system.indexes']

    for CollectionName in CollectionNames:
        print('CollectionName', CollectionName)
        RecordsList = [rec for rec in con[RouteName][CollectionName].find().sort([('_id',1)])]

        for RecordDict in RecordsList:
            del[RecordDict['_id']]

        if os.path.exists(os.path.join(NpPathDir, RouteName)) == False:
            os.mkdir(os.path.join(NpPathDir, RouteName))

            
        #np.save(f'{Path}/{Database}/{CollectionName}.npy', RecordsList)
        np.save(os.path.join(NpPathDir, RouteName,f'{CollectionName}.npy'), RecordsList)
        

## Execute the preprocessing

### Parameters

In [12]:
path = Path(os.getcwd())
OneLevelUpPath = path.parents[0]

In [13]:
IntervalLength = 160

'''Trips and Window size for Feature extraction'''
#BMSingleTripsInfoList,CMSingleTripsInfoList,PDSingleTripsInfoList = TransportModeFeatureHelper.GetTripsBasedOnType(RouteName)
WindowList = [32,64,128,256,512]
WindowIndex = 2
WindowSize = WindowList[WindowIndex]

'''For segment other than stoppages'''
Speed_H = 5.6
Speed_L = 3

'''Path for Np'''
NpPathDir = os.path.join(str(OneLevelUpPath), 'data','NpData')

## Variables

`ProjectDataUsed`: determines whether the project data or the user's own data is used for execution.

`UsedPreTrained`: determines whether the pretrained and precomputed dataset or raw data is used for execution.

`UseMongoDB`: determines whether the MonngoDB database or Numpy file is used for execution.

`ReducedKFolds`: If false: one fold is used, else ten-fold is used

In [15]:
'''
ProjectDataUsed = True
UsedPreTrained = True
ReducedKFolds = False
UseMongoDB = True
'''

#'''
ProjectDataUsed = True
UsedPreTrained = True
ReducedKFolds = True
UseMongoDB = False
#'''

'''
ProjectDataUsed = True
UsedPreTrained = False
ReducedKFolds = False
UseMongoDB = False
'''

In [ ]:
#SitStandRecordDir = '/home/pruthvish/JRF/GitVersion_PMC/Data/SitStandRecord'
if ProjectDataUsed==True:
    SitStandRecordDir = os.path.join(str(OneLevelUpPath), 'data','SitStandRecord','')
else:
    SitStandRecordDir = os.path.join(str(OneLevelUpPath), 'data','UserData','SitStandRecord','')
#RouteNamesList = [f for f in os.listdir(SitStandRecordDir)]
RouteNamesList = ['HAR_PDPU_SANAND_ ', 'HAR_PDPU_SANAND_Triggers_']

### Code

In [16]:
for RouteName in RouteNamesList:
    print(RouteName)
    if UsedPreTrained==False and UseMongoDB==True:
        print('Reading data and saving in MongoDB')
        ReadAndSaveData(SitStandRecordDir, RouteName)
        
        RecordType = '.Raw'
        print(f'Computing preprocessing for {RecordType} segments')
        ComputeEarthAxisComponentRaw(RouteName, IntervalLength, RecordType)
        print(f'Computing features for {RecordType} segments')
        ComputeFeatures_Raw(ProjectDataUsed, WindowList, WindowIndex, RecordType, RouteName)
        
        if RouteName == 'HAR_PDPU_SANAND_':
            RecordType = '.SegmentOtherThanStoppage'
            print(f'Computing preprocessing for {RecordType} segments')
            ExtractSegment_EarthAxis_OtherThanStoppages(RouteName, Speed_H , Speed_L)
            print(f'Computing features for {RecordType} segments')
            ComputeFeatures_SegmentOtherThanStoppages_SitStand(ProjectDataUsed, WindowList, WindowIndex, 
                                                      RecordType, RouteName)
        
        print('Saving MongoData in Np files')
        SaveInNp(RouteName, NpPathDir)
    
    elif UseMongoDB==True:
        RouteNamesListInDB = con.list_database_names()
        if RouteName not in RouteNamesListInDB:
            '''Load the data for RouteName, if RouteName is not in RouteNamesList'''
            print('Loading MongoData from Np files')
            LoadInMongoFromNp(RouteName, NpPathDir)


HAR_PDPU_SANAND_Triggers_
HAR_PDPU_SANAND_


## Classification

In [17]:
'''Import Libs'''
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix,precision_score, recall_score, f1_score
from sklearn import tree
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from collections import Counter
from sklearn.impute import SimpleImputer

In [18]:
'''Variables for Classification'''
ResultPathDir = os.path.join(str(OneLevelUpPath), 'results','SitStand','')

if os.path.exists(ResultPathDir) == False:
    os.mkdir(ResultPathDir)

TrainedModelPathDir = os.path.join(str(OneLevelUpPath), 'data', 'TrainedModel','SitStand','')
'''Any one route type as decided by Stoppage experiment'''
#RecordType = '.SegmentOtherThanStoppage'
#RecordType = '.StoppageSegment'
RecordType = '.Raw'

ClassifierList = [GaussianNB(), LogisticRegression(random_state=0), 
                  RandomForestClassifier(max_depth=20), tree.DecisionTreeClassifier(), 
                  SVC(gamma='auto')
                 ]
ClassifierNameList = ['NB', 'LogisticRegression', 'RF', 'DT', 'SVC']

In [19]:
def GetFeaturesForClass(RouteNameTestList, FeatureType, RecordType,
                        SelectedFeatures, SelectedFeaturesFlag, NpPathDir, UseMongoDB):
    '''
    if ProjectDataUsed==True:
        X, y = TMC_Helper.GetData(FeatureType, RecordType, SelectedFeatures, SelectedFeaturesFlag)
    else:
    '''
    #X = []
    #y = []
    if UseMongoDB==True:
        First_InputFlag = True

        for RouteNameTest in RouteNameTestList:
            X_Route, y_Route = CAC_Helper.GetData_User(RouteNameTest, FeatureType, RecordType,
                                                       SelectedFeatures, SelectedFeaturesFlag)

            if First_InputFlag==True:
                X = X_Route
                y = y_Route
                First_InputFlag = False

            else:
                X = np.concatenate((X, X_Route))
                y = np.concatenate((y, y_Route))
                '''
                X += X_Route
                y += y_Route
                '''

            print('X.shape', X.shape)
            print('y.shape', y.shape)
        
        '''Save in NpData'''
        
        if os.path.exists(os.path.join(NpPathDir,'SitStand'))==False:
            os.mkdir(os.path.join(NpPathDir,'SitStand'))
            
        np.save(f'{NpPathDir}/SitStand/X_{FeatureType}_{RecordType}_{SelectedFeaturesFlag}.npy', X)
        np.save(f'{NpPathDir}/SitStand/y_{FeatureType}_{RecordType}_{SelectedFeaturesFlag}.npy', y)
        
    else:
        X = np.load(f'{NpPathDir}/SitStand/X_{FeatureType}_{RecordType}_{SelectedFeaturesFlag}.npy',
                    allow_pickle=True)
        y = np.load(f'{NpPathDir}/SitStand/y_{FeatureType}_{RecordType}_{SelectedFeaturesFlag}.npy',
                    allow_pickle=True)
    
    return(X, y)

In [20]:
def ApplyClassification(Classes, FeatureType, RecordType, SelectedFeatures, SelectedFeaturesFlag,
                        Classifier, ClassifierName, ResultPathDir, ProjectDataUsed, RouteNameTestList, 
                        UsedPreTrained, ClassifierName_ForModelSave, TrainedModelPathDir, ReducedKFolds, 
                        NpPathDir, UseMongoDB
                       ):
    
    '''
    input: 
        Classes: The number of classes
        
        FeatureType and RecordType: feature type and record type information
        
        SelectedFeatures and SelectedFeaturesFlag: selected features list and flag to 
        determine if selected features are list is used
        
        Classifier and ClassifierName: classifier object variable and classifier name
        
        ResultPathDir: path of result directory
        
        ProjectDataUsed: flag to determine whether the project dataset or user dataset is used
        
        RouteNameTestList: The route names list to be considered during classification
        
        UsedPreTrained: Flag to determine whether the pretrained classifier should be used
        or the classifier is to be trained
        
        ClassifierName_ForModelSave: the name of classifier for saving a model
        
        TrainedModelPathDir: path of trainedModel directory 
        
        NpPathDir: 
        
        UseMongoDB: 
    
    output: None
    
    function: It trains and validates the classifier the ten-fold cross-validation with stratified
    sampling of each class. The performance metrics of the classifier is stores as a txt file in the
    result directory
    
    '''    
    
    MetricsDict = CAC_Helper.InitializeMetricsDict(Classes)
    
    
    X, y = GetFeaturesForClass(RouteNameTestList, FeatureType, RecordType,
                               SelectedFeatures, SelectedFeaturesFlag, NpPathDir, UseMongoDB)    
    
    
    MetricsDict = CAC_Helper.TrainAndPredict(X, y, Classifier, MetricsDict, ResultPathDir,
                                             ClassifierName, UsedPreTrained, 
                                             ClassifierName_ForModelSave, TrainedModelPathDir, ReducedKFolds)
    CAC_Helper.PrintMetricsDict(ClassifierName, ResultPathDir, FeatureType, RecordType,
                                SelectedFeaturesFlag, MetricsDict)
    
    CAC_Helper.InfereSitStand_And_PrintMetrics(MetricsDict, ClassifierName, ResultPathDir)
    
    '''
    filename = 'finalized_model.sav'
    pickle.dump(model, open(ResultPathDir, 'wb'))
    '''


In [21]:
Classes = 5

In [22]:
for Classifier, ClassifierName in zip(ClassifierList, ClassifierNameList):
    print('Classifier, ClassifierName', Classifier, ClassifierName)
    #for RecordType in ['.Raw', '.SegmentOtherThanStoppage']:

    '''Feature set-1'''
    FeatureType = '.HARFeature'
    SelectedFeaturesFlag = False
    SelectedFeatures = []

    print('Feature set-1')
    ClassifierName_ForModelSave = ClassifierName+"_Set1"
    ApplyClassification(Classes, FeatureType, RecordType, SelectedFeatures, SelectedFeaturesFlag,
                        Classifier, ClassifierName, ResultPathDir, ProjectDataUsed, RouteNamesList,
                        UsedPreTrained, ClassifierName_ForModelSave, TrainedModelPathDir, ReducedKFolds, 
                        NpPathDir, UseMongoDB
                       )

    '''Feature set-2'''
    FeatureType = '.HARFeature'
    SelectedFeaturesFlag = True
    ClassifierName_ForModelSave = ClassifierName+"_Set2"
    SelectedFeatures = CAC_Helper.SelectedFeaturesForFeatureType(FeatureType)

    print('Feature set-2')

    ApplyClassification(Classes, FeatureType, RecordType, SelectedFeatures, SelectedFeaturesFlag,
                        Classifier, ClassifierName, ResultPathDir, ProjectDataUsed, RouteNamesList, 
                        UsedPreTrained, ClassifierName_ForModelSave, TrainedModelPathDir, ReducedKFolds, 
                        NpPathDir, UseMongoDB
                       )

    '''Feature set-3'''
    FeatureType = '.TransportFeatures'
    SelectedFeatures = []
    SelectedFeaturesFlag = False
    print('Feature set-3')
    ClassifierName_ForModelSave = ClassifierName+"_Set3"

    ApplyClassification(Classes, FeatureType, RecordType, SelectedFeatures, SelectedFeaturesFlag,
                        Classifier, ClassifierName, ResultPathDir, ProjectDataUsed, RouteNamesList,
                        UsedPreTrained, ClassifierName_ForModelSave, TrainedModelPathDir, ReducedKFolds,
                        NpPathDir, UseMongoDB
                       )   


    '''Feature set-4'''
    FeatureType = '.TransportFeatures'
    SelectedFeatures = CAC_Helper.SelectedFeaturesForFeatureType(FeatureType)
    SelectedFeaturesFlag = True

    print('Feature set-4')
    ClassifierName_ForModelSave = ClassifierName+"_Set4"
    ApplyClassification(Classes, FeatureType, RecordType, SelectedFeatures, SelectedFeaturesFlag,
                        Classifier, ClassifierName, ResultPathDir, ProjectDataUsed, RouteNamesList,
                        UsedPreTrained, ClassifierName_ForModelSave, TrainedModelPathDir, ReducedKFolds, 
                        NpPathDir, UseMongoDB
                       )        



Classifier, ClassifierName GaussianNB() NB
Feature set-1
Feature set-2
Feature set-3
Feature set-4
Classifier, ClassifierName LogisticRegression(random_state=0) LogisticRegression
Feature set-1


/home/pruthvish/JRF/RoadNetwork/RoadNetwork_VirtualEnv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/pruthvish/JRF/RoadNetwork/RoadNetwork_VirtualEnv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https:/

Feature set-2


/home/pruthvish/JRF/RoadNetwork/RoadNetwork_VirtualEnv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/pruthvish/JRF/RoadNetwork/RoadNetwork_VirtualEnv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https:/

Feature set-3


/home/pruthvish/JRF/RoadNetwork/RoadNetwork_VirtualEnv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/pruthvish/JRF/RoadNetwork/RoadNetwork_VirtualEnv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https:/

Feature set-4


/home/pruthvish/JRF/RoadNetwork/RoadNetwork_VirtualEnv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/pruthvish/JRF/RoadNetwork/RoadNetwork_VirtualEnv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https:/

Classifier, ClassifierName RandomForestClassifier(max_depth=20) RF
Feature set-1
Feature set-2
Feature set-3
Feature set-4
Classifier, ClassifierName DecisionTreeClassifier() DT
Feature set-1
Feature set-2
Feature set-3
Feature set-4
Classifier, ClassifierName SVC(gamma='auto') SVC
Feature set-1
Feature set-2
Feature set-3
Feature set-4


In [23]:
'''Read the value for one of the machine learning algorithm'''
file = os.path.join(ResultPathDir,f'{ClassifierName}.txt')
f = open(file, "r")
print(f.read())

Results for .Raw, .HARFeature, and selected features flag: False
ConfusionMatrix 
[[845.   0.   4.  24.   5.]
 [ 12. 902.   0.   5.   3.]
 [  1.   0. 859.   5.   1.]
 [111.   0.  26. 714.   1.]
 [ 78.   5.   6.  23. 309.]]
 
PrecissionValue 
[0.80706781 0.99448732 0.95977654 0.92607004 0.96865204]
 
RecallValue 
[0.96241458 0.97830803 0.99191686 0.83802817 0.73396675]
 
F1ScoreValue 
[0.87792208 0.98633133 0.97558206 0.87985213 0.83513514]
 
AccuracyValue 
0.9212998222899214
 
Inference for sit-stand 
SitStand_ConfusionMetrics 
[[1759.   41.]
 [ 195. 1944.]]
 
PrecissionMetrics 
[0.90020471 0.97934509]
 
RecallMetrics 
[0.97722222 0.9088359 ]
 
Accuracy 
0.94008632
 
Results for .Raw, .HARFeature, and selected features flag: True
ConfusionMatrix 
[[831.   2.   4.  32.   9.]
 [ 19. 887.   0.  12.   4.]
 [  4.   0. 855.   6.   1.]
 [150.   0.  25. 676.   1.]
 [ 96.   6.   3.  28. 288.]]
 
PrecissionValue 
[0.75545455 0.99106145 0.96392334 0.89655172 0.95049505]
 
RecallValue 
[0.94646925